In [1]:
from env import config
import requests

In [2]:
# MINDSDB_UN=config("MINDSDB_UN", default=None)# abc@mindsdb.com
# MINDSDB_PW=config("MINDSDB_PW", default=None) # xxxxxxxx
# assert MINDSDB_UN is not None
# assert MINDSDB_PW is not None


# MINDSDB_BASE_URL = "https://cloud.mindsdb.com/api"

In [3]:
# def get_mindsdb_session():
#     session = requests.Session()
#     session.post('https://cloud.mindsdb.com/cloud/login', json={
#         'email': MINDSDB_UN,
#         'password': MINDSDB_PW
#     })
#     return session



In [4]:
MINDSDB_BASE_URL = 'http://127.0.0.1:47334/api'

In [5]:
# def mindsdb_query(session, sql_query):
#     endpoint = "/sql/query"
#     url = f"{MINDSDB_BASE_URL}{endpoint}"
#     headers = {"Content-Type": "application/json"}
#     return session.post(url, json={"query": sql_query}, headers=headers)

In [6]:


# # ['flightDate',
# #  'startingAirport',
# #  'destinationAirport',
# #  'isBasicEconomy',
# #  'isRefundable',
# #  'isNonStop',
# #  'segmentsAirlineName',
# #  'totalFare',
# #  '__mdb_ts_previous_totalFare',
# #  'select_data_query',
# #  'when_data',
# #  'totalFare_original',
# #  'totalFare_confidence',
# #  'totalFare_explain',
# #  'totalFare_anomaly',
# #  'totalFare_min',
# #  'totalFare_max',
 
# def predict_query(session, 
#         flightDate = "2022-04-21", 
#         startingAirport="SFO", 
#         isNonStop=1, 
#         destinationAirport="BOS",
#         raw_request=False,
#         ):
#     sql_query = f"""
#     SELECT m.flightDate, m.segmentsAirlineName, m.isNonStop, m.totalFare, m.totalFare_confidence FROM mindsdb.flight_price_predictor AS m
#     JOIN ai_travel_agent.flight_prices AS t
#     WHERE t.flightDate = "{flightDate}"
#     AND t.startingAirport = "{startingAirport}"
#     AND t.isNonStop = {isNonStop}
#     AND t.destinationAirport = "{destinationAirport}";
#     """
#     response = mindsdb_query(session, sql_query)
#     response.raise_for_status()
#     if raw_request:
#         return response
#     data = response.json()
#     columns = data.get("column_names")
#     dataset = data.get('data')
#     web_ready_data = [dict(zip(columns, row)) for row in dataset]
#     return web_ready_data
        
    



In [7]:
# def mindsdb_query(sql_query):
#     endpoint = "/sql/query"
#     url= f"{MINDSDB_BASE_URL}{endpoint}"
#     headers = {"Content-Type": "application/json"}
#     response = requests.post(
#         url,
#         json={"query": sql_query},
#         headers=headers
#     )
#     return response

In [8]:
def mindsdb_query(sql_query: str):
    endpoint = "/sql/query"
    url = f"{MINDSDB_BASE_URL}{endpoint}"

    headers = {"Content-Type": "application/json"}

    try:
        response = requests.post(
            url,
            json={"query": sql_query},
            headers=headers,
            timeout=10,
        )
        response.raise_for_status()
        return response.json()

    except requests.exceptions.RequestException as e:
        print("MindsDB request failed:", e)
        return None


In [9]:
# def predict_query(
#     flightDate: str = "2022-04-21",
#     startingAirport: str = "SFO",
#     isNonStop: int = 1,
#     destinationAirport: str = "BOS",
#     raw_request: bool = True,
# ):
#     sql_query = f"""
#         SELECT m.flightDate, m.segmentsAirlineName, m.isNonStop, m.totalFare, m.totalFare_confidence
#         FROM mindsdb.flight_price_predictor AS m
#         JOIN ai_travel_agent.ai_travel_prices.flight_prices AS t
#         WHERE t.flightDate > '{flightDate}'
#         AND t.startingAirport = '{startingAirport}'
#         AND t.isNonStop = {isNonStop}
#         AND t.destinationAirport = '{destinationAirport}' LIMIT 5;
#     """
#     print(sql_query)

#     response = mindsdb_query(sql_query)
#     response.raise_for_status()

#     if raw_request:
#         return response

#     data = response.json()
#     columns = data.get("column_names")
#     dataset = data.get("data")
#     web_ready_data = [dict(zip(columns, row)) for row in dataset]

#     return web_ready_data


In [10]:
def predict_query(
    flightDate="2022-04-21",
    startingAirport="SFO",
    isNonStop=1,
    isBasicEconomy=0,
    isRefundable=0,
    destinationAirport="BOS",
    raw_request=False,
    **kwargs,

):
    sql_query = f"""
        SELECT m.flightDate as date, m.segmentsAirlineName as airline, m.isNonStop as nonStop, m.isBasicEconomy as basic, m.isRefundable as refundable,
               m.totalFare as price
        FROM mindsdb.flight_price_predictor AS m
        JOIN ai_travel_agent.ai_travel_prices.flight_prices AS t
        WHERE t.flightDate > '{flightDate}'
        AND t.startingAirport = '{startingAirport}'
        AND t.isBasicEconomy = {isBasicEconomy}
        AND t.isRefundable = {isRefundable}
        AND t.isNonStop = {isNonStop}
        AND t.destinationAirport = '{destinationAirport}' LIMIT 5;
    """

    print("Executing SQL:")
    print(sql_query)

    result = mindsdb_query(sql_query)

    if result is None:
        raise Exception("No response received from MindsDB")

    if raw_request:
        return result

    columns = result.get("column_names")
    dataset = result.get("data")

    if not columns or not dataset:
        return []

    web_ready_data = [
        dict(zip(columns, row))
        for row in dataset
    ]

    return web_ready_data

In [11]:
query_response = predict_query(
    flightDate="2022-04-21",
    startingAirport="SFO",
    isNonStop=1,
    destinationAirport="BOS",
    raw_request=False,
)


Executing SQL:

        SELECT m.flightDate as date, m.segmentsAirlineName as airline, m.isNonStop as nonStop, m.isBasicEconomy as basic, m.isRefundable as refundable,
               m.totalFare as price
        FROM mindsdb.flight_price_predictor AS m
        JOIN ai_travel_agent.ai_travel_prices.flight_prices AS t
        WHERE t.flightDate > '2022-04-21'
        AND t.startingAirport = 'SFO'
        AND t.isBasicEconomy = 0
        AND t.isRefundable = 0
        AND t.isNonStop = 1
        AND t.destinationAirport = 'BOS' LIMIT 5;
    


In [12]:
query_response

[{'date': '2022-04-23 00:00:00.000000',
  'airline': 'Alaska Airlines',
  'nonStop': '1',
  'basic': '0',
  'refundable': '0',
  'price': 132.94475820166434},
 {'date': '2022-04-25 00:00:00.000000',
  'airline': 'Alaska Airlines',
  'nonStop': '1',
  'basic': '0',
  'refundable': '0',
  'price': 152.64437265037816},
 {'date': '2022-04-27 00:00:00.000000',
  'airline': 'Alaska Airlines',
  'nonStop': '1',
  'basic': '0',
  'refundable': '0',
  'price': 133.89045721600004},
 {'date': '2022-04-29 00:00:00.000000',
  'airline': 'Alaska Airlines',
  'nonStop': '1',
  'basic': '0',
  'refundable': '0',
  'price': 26.206964930710697},
 {'date': '2022-05-01 00:00:00.000000',
  'airline': 'Alaska Airlines',
  'nonStop': '1',
  'basic': '0',
  'refundable': '0',
  'price': 30.458091381765154}]

In [13]:
# data = query_response.json()
# data.keys()

In [14]:
# columns = data.get("column_names")
# dataset = data.get("data")
# columns

In [15]:
# dataset

In [16]:
# web_ready_data = []
# for row in dataset:
#     print(row)
#     for i,col in enumerate(columns):
#         print(columns[i],col)

In [49]:
# web_ready_data = [dict(zip(columns, row)) for row in dataset]

In [50]:
# web_ready_data

In [2]:
# Check if MindsDB is accessible
# curl http://127.0.0.1:47334/api/status

In [ ]:
# import requests

# api_key = ""

# headers = {
#     "x-api-key": api_key,  # NOT "Authorization: Bearer"
#     "anthropic-version": "2023-06-01",
#     "content-type": "application/json"
# }

# data = {
#     "model": "claude-sonnet-4-20250514",
#     "max_tokens": 100,
#     "messages": [{"role": "user", "content": "Say hello"}]
# }

# response = requests.post(
#     "https://api.anthropic.com/v1/messages",
#     headers=headers,
#     json=data
# )

# print(response.json())

In [19]:
import json
pref = {
    "flightDate": "2022-04-21",
    "startingAirport": "SFO",
    "isNonStop": 1,
    "destinationAirport": "BOS",
}

# If query_response is a DataFrame:
options = query_response

rec_context = json.dumps({
    "options": options,
    "preferences": pref
})
rec_context

'{"options": [{"date": "2022-04-23 00:00:00.000000", "airline": "Alaska Airlines", "nonStop": "1", "basic": "0", "refundable": "0", "price": 132.94475820166434}, {"date": "2022-04-25 00:00:00.000000", "airline": "Alaska Airlines", "nonStop": "1", "basic": "0", "refundable": "0", "price": 152.64437265037816}, {"date": "2022-04-27 00:00:00.000000", "airline": "Alaska Airlines", "nonStop": "1", "basic": "0", "refundable": "0", "price": 133.89045721600004}, {"date": "2022-04-29 00:00:00.000000", "airline": "Alaska Airlines", "nonStop": "1", "basic": "0", "refundable": "0", "price": 26.206964930710697}, {"date": "2022-05-01 00:00:00.000000", "airline": "Alaska Airlines", "nonStop": "1", "basic": "0", "refundable": "0", "price": 30.458091381765154}], "preferences": {"flightDate": "2022-04-21", "startingAirport": "SFO", "isNonStop": 1, "destinationAirport": "BOS"}}'

In [20]:
rec_sql_query = f"""
SELECT answer
FROM ai_travel_agent
WHERE question = 'Respond with JSON Only with given column names, what is the best flight option?'
AND context = '{rec_context}'
LIMIT 1;
"""
rec_sql_query

'\nSELECT answer\nFROM ai_travel_agent\nWHERE question = \'Respond with JSON Only with given column names, what is the best flight option?\'\nAND context = \'{"options": [{"date": "2022-04-23 00:00:00.000000", "airline": "Alaska Airlines", "nonStop": "1", "basic": "0", "refundable": "0", "price": 132.94475820166434}, {"date": "2022-04-25 00:00:00.000000", "airline": "Alaska Airlines", "nonStop": "1", "basic": "0", "refundable": "0", "price": 152.64437265037816}, {"date": "2022-04-27 00:00:00.000000", "airline": "Alaska Airlines", "nonStop": "1", "basic": "0", "refundable": "0", "price": 133.89045721600004}, {"date": "2022-04-29 00:00:00.000000", "airline": "Alaska Airlines", "nonStop": "1", "basic": "0", "refundable": "0", "price": 26.206964930710697}, {"date": "2022-05-01 00:00:00.000000", "airline": "Alaska Airlines", "nonStop": "1", "basic": "0", "refundable": "0", "price": 30.458091381765154}], "preferences": {"flightDate": "2022-04-21", "startingAirport": "SFO", "isNonStop": 1, "d

In [21]:
 rec_prediction_response = mindsdb_query(rec_sql_query)

In [22]:
rec_prediction_response

{'type': 'table',
 'data': [['```json\n{\n  "date": "2022-04-29 00:00:00.000000",\n  "airline": "Alaska Airlines", \n  "nonStop": "1",\n  "basic": "0",\n  "refundable": "0",\n  "price": 26.206964930710697\n}\n```']],
 'column_names': ['answer'],
 'context': {'show_secrets': False, 'db': 'mindsdb'}}

In [23]:
# SELECT answer FROM ai_travel_agent
# WHERE question = 'what is bigger a or b?'
#   AND context = 'b is 10 and a is 5';

In [24]:
import re

In [25]:
def recommended_flight(
    user_data={},
    forecast_data=[],
    question="Respond with JSON Only with given column names, what is the best flight option?",
    raw_request=False,
    **kwargs,
):
    context = {"options": forecast_data, "preferences": user_data}
    context_data = json.dumps(context)

    # Escape single quotes to prevent SQL break
    safe_context = context_data.replace("'", "''")
    safe_question = question.replace("'", "''")

    sql_query = f"""
    SELECT answer
    FROM ai_travel_agent
    WHERE question = '{safe_question}'
    AND context = '{safe_context}'
    LIMIT 1;
    """

    # print("Executing SQL:")
    # print(sql_query)

    result = mindsdb_query(sql_query)

    if result is None:
        raise Exception("No response received from MindsDB")

    if raw_request:
        return result

    columns = result.get("column_names")
    dataset = result.get("data")

    if not columns or not dataset:
        return []

    answer = dataset[0][0]

    json_match = re.search(r'\{.*\}', answer, re.DOTALL)
    
    if json_match:
        return json_match.group(0)
    
    return answer


In [26]:
response = recommended_flight(user_data=pref, forecast_data=query_response)
print("RAW RESPONSE:")
print(response)
print("TYPE:", type(response))


RAW RESPONSE:
{
  "date": "2022-04-29 00:00:00.000000",
  "airline": "Alaska Airlines", 
  "nonStop": "1",
  "basic": "0",
  "refundable": "0",
  "price": 26.206964930710697
}
TYPE: <class 'str'>


In [27]:
json.loads(
    recommended_flight(user_data=pref, forecast_data=query_response)
)


{'date': '2022-04-29 00:00:00.000000',
 'airline': 'Alaska Airlines',
 'nonStop': '1',
 'basic': '0',
 'refundable': '0',
 'price': 26.206964930710697}